In [1]:
import pandas as pd
import datetime as dt
from datetime import date,time,datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
lbl = LabelEncoder()
mlb = MultiLabelBinarizer()
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_pickle("./dataset.pkl")

In [3]:
df

,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,50808,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
0,2,0,65,462.0,0.0,24.0,22.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,1,1,0
1,6,1,71,462.0,0.0,24.0,26.0,1.0,104.0,1.20,...,0,0,0,0,0,0,0,0,0,1
2,7,1,75,462.0,1.0,24.0,27.0,1.0,98.0,1.07,...,0,0,0,0,0,1,0,0,0,1
3,3,1,39,462.0,0.0,24.0,27.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,0,0,0
4,4,1,59,462.0,0.0,24.0,26.0,1.0,104.0,1.11,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,3,0,20,230.0,-2.0,26.0,25.0,1.0,101.0,1.12,...,0,0,0,0,0,0,0,1,1,0
58972,1,0,84,485.0,-3.0,18.0,18.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,0,0,1
58973,9,1,42,422.0,-1.0,24.0,27.0,1.0,103.0,1.08,...,0,1,0,1,0,0,0,1,1,1
58974,11,0,60,462.0,-3.0,22.0,23.0,2.0,111.0,1.00,...,0,0,0,0,0,0,0,1,1,0


In [4]:
x = df.iloc[:,:483]
x.shape
#483 = 3(adm_days,gender,age)+480(lab values)

(58976, 483)

In [5]:
y = df.iloc[:,483:]


In [6]:
y.columns

Index(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15)

In [8]:
y.shape

(58976, 20)

In [9]:
x_train.shape

(50129, 483)

In [10]:
y_train.shape

(50129, 20)

In [11]:
x_test.shape

(8847, 483)

In [12]:
y_test.shape

(8847, 20)

In [13]:
np.unique(y_train)

array([0, 1])

In [25]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
'''sample_weights = [1-3032/58976,1-8924/58976,1-35608/58976,1-19366/58976,
                 1-16271/58976,1-20008/58976,1-43243/58976,
                 1-25281/58976,1-21200/58976,1-21782/58976,1-166/58976,          this for xgb and lgbm
                 1-8316/58976,1-10053/58976,1-3205/58976,
                 1-5492/58976,1-19454/58976,1-1757/58976,1-23352/58976,1-16621/58976,1-34350/58976]'''
weights = [0.286,0.714]#this for catboost
#[0,1] classes 0.286 is probabbility of 1 in dataset(icd) so i flipped it over as to give more weight to 1

#model1 = CatBoostClassifier(iterations=1000, task_type="GPU",devices='0:1',learning_rate=0.1)
#d_train = lgb.Dataset(x_train, label=y_train)
#clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=100))
clf_multilabel = OneVsRestClassifier(CatBoostClassifier(iterations=1000,learning_rate=0.15,class_weights=weights))

In [26]:
'''parameters = {
    
    
    "learning_rate":[0.1, 0.2, 0.3, 0.4]
}

model_tunning = GridSearchCV(clf_multilabel, param_grid=parameters,n_jobs=-1,cv=10,
                             scoring = 'roc_auc')

model_tunning.fit(x_train, y_train)

print(model_tunning.best_score_)
print(model_tunning.best_params_)'''

'parameters = {\n    \n    \n    "learning_rate":[0.1, 0.2, 0.3, 0.4]\n}\n\nmodel_tunning = GridSearchCV(clf_multilabel, param_grid=parameters,n_jobs=-1,cv=10,\n                             scoring = \'roc_auc\')\n\nmodel_tunning.fit(x_train, y_train)\n\nprint(model_tunning.best_score_)\nprint(model_tunning.best_params_)'

In [27]:
#estimator.get_params().keys()

In [28]:
clf_multilabel.fit(x_train,y_train)


0:	learn: 0.5752324	total: 207ms	remaining: 3m 26s
1:	learn: 0.4957894	total: 360ms	remaining: 2m 59s
2:	learn: 0.4433608	total: 491ms	remaining: 2m 43s
3:	learn: 0.4039004	total: 625ms	remaining: 2m 35s
4:	learn: 0.3800241	total: 745ms	remaining: 2m 28s
5:	learn: 0.3629259	total: 868ms	remaining: 2m 23s
6:	learn: 0.3455409	total: 1s	remaining: 2m 22s
7:	learn: 0.3363564	total: 1.14s	remaining: 2m 21s
8:	learn: 0.3292718	total: 1.28s	remaining: 2m 21s
9:	learn: 0.3238137	total: 1.42s	remaining: 2m 20s
10:	learn: 0.3198983	total: 1.55s	remaining: 2m 19s
11:	learn: 0.3157342	total: 1.68s	remaining: 2m 18s
12:	learn: 0.3126839	total: 1.81s	remaining: 2m 17s
13:	learn: 0.3100887	total: 1.93s	remaining: 2m 16s
14:	learn: 0.3084126	total: 2.06s	remaining: 2m 15s
15:	learn: 0.3065543	total: 2.19s	remaining: 2m 15s
16:	learn: 0.3049788	total: 2.33s	remaining: 2m 14s
17:	learn: 0.3034711	total: 2.46s	remaining: 2m 14s
18:	learn: 0.3020715	total: 2.6s	remaining: 2m 14s
19:	learn: 0.3007847	total

160:	learn: 0.2390290	total: 23.5s	remaining: 2m 2s
161:	learn: 0.2386247	total: 23.7s	remaining: 2m 2s
162:	learn: 0.2382705	total: 23.8s	remaining: 2m 2s
163:	learn: 0.2378501	total: 24s	remaining: 2m 2s
164:	learn: 0.2376462	total: 24.2s	remaining: 2m 2s
165:	learn: 0.2371921	total: 24.3s	remaining: 2m 2s
166:	learn: 0.2371498	total: 24.4s	remaining: 2m 1s
167:	learn: 0.2367926	total: 24.6s	remaining: 2m 1s
168:	learn: 0.2363669	total: 24.8s	remaining: 2m 1s
169:	learn: 0.2359127	total: 24.9s	remaining: 2m 1s
170:	learn: 0.2356513	total: 25s	remaining: 2m 1s
171:	learn: 0.2354845	total: 25.2s	remaining: 2m 1s
172:	learn: 0.2350948	total: 25.4s	remaining: 2m 1s
173:	learn: 0.2348464	total: 25.5s	remaining: 2m 1s
174:	learn: 0.2345214	total: 25.7s	remaining: 2m 1s
175:	learn: 0.2342090	total: 25.9s	remaining: 2m 1s
176:	learn: 0.2341621	total: 26s	remaining: 2m
177:	learn: 0.2337930	total: 26.2s	remaining: 2m
178:	learn: 0.2337536	total: 26.3s	remaining: 2m
179:	learn: 0.2333301	total

318:	learn: 0.1965006	total: 46.8s	remaining: 1m 39s
319:	learn: 0.1962047	total: 46.9s	remaining: 1m 39s
320:	learn: 0.1958982	total: 47.1s	remaining: 1m 39s
321:	learn: 0.1957532	total: 47.2s	remaining: 1m 39s
322:	learn: 0.1954794	total: 47.4s	remaining: 1m 39s
323:	learn: 0.1952195	total: 47.5s	remaining: 1m 39s
324:	learn: 0.1949791	total: 47.7s	remaining: 1m 39s
325:	learn: 0.1947106	total: 47.8s	remaining: 1m 38s
326:	learn: 0.1944185	total: 48s	remaining: 1m 38s
327:	learn: 0.1942104	total: 48.2s	remaining: 1m 38s
328:	learn: 0.1939630	total: 48.3s	remaining: 1m 38s
329:	learn: 0.1937519	total: 48.4s	remaining: 1m 38s
330:	learn: 0.1935538	total: 48.6s	remaining: 1m 38s
331:	learn: 0.1931084	total: 48.7s	remaining: 1m 37s
332:	learn: 0.1929861	total: 48.8s	remaining: 1m 37s
333:	learn: 0.1926031	total: 49s	remaining: 1m 37s
334:	learn: 0.1921740	total: 49.1s	remaining: 1m 37s
335:	learn: 0.1918107	total: 49.3s	remaining: 1m 37s
336:	learn: 0.1915109	total: 49.5s	remaining: 1m 3

474:	learn: 0.1660536	total: 1m 9s	remaining: 1m 16s
475:	learn: 0.1658932	total: 1m 9s	remaining: 1m 16s
476:	learn: 0.1657192	total: 1m 9s	remaining: 1m 16s
477:	learn: 0.1654839	total: 1m 9s	remaining: 1m 15s
478:	learn: 0.1653140	total: 1m 9s	remaining: 1m 15s
479:	learn: 0.1651464	total: 1m 9s	remaining: 1m 15s
480:	learn: 0.1649509	total: 1m 9s	remaining: 1m 15s
481:	learn: 0.1647944	total: 1m 10s	remaining: 1m 15s
482:	learn: 0.1646227	total: 1m 10s	remaining: 1m 15s
483:	learn: 0.1645991	total: 1m 10s	remaining: 1m 14s
484:	learn: 0.1644553	total: 1m 10s	remaining: 1m 14s
485:	learn: 0.1643875	total: 1m 10s	remaining: 1m 14s
486:	learn: 0.1642620	total: 1m 10s	remaining: 1m 14s
487:	learn: 0.1640885	total: 1m 10s	remaining: 1m 14s
488:	learn: 0.1639827	total: 1m 10s	remaining: 1m 14s
489:	learn: 0.1636840	total: 1m 11s	remaining: 1m 13s
490:	learn: 0.1634818	total: 1m 11s	remaining: 1m 13s
491:	learn: 0.1633725	total: 1m 11s	remaining: 1m 13s
492:	learn: 0.1630893	total: 1m 11s

630:	learn: 0.1406282	total: 1m 32s	remaining: 54s
631:	learn: 0.1404750	total: 1m 32s	remaining: 53.9s
632:	learn: 0.1403594	total: 1m 32s	remaining: 53.8s
633:	learn: 0.1402348	total: 1m 32s	remaining: 53.6s
634:	learn: 0.1400746	total: 1m 33s	remaining: 53.5s
635:	learn: 0.1399420	total: 1m 33s	remaining: 53.3s
636:	learn: 0.1398654	total: 1m 33s	remaining: 53.2s
637:	learn: 0.1396789	total: 1m 33s	remaining: 53.1s
638:	learn: 0.1394510	total: 1m 33s	remaining: 52.9s
639:	learn: 0.1392650	total: 1m 33s	remaining: 52.8s
640:	learn: 0.1391899	total: 1m 33s	remaining: 52.6s
641:	learn: 0.1390426	total: 1m 34s	remaining: 52.5s
642:	learn: 0.1388953	total: 1m 34s	remaining: 52.3s
643:	learn: 0.1387225	total: 1m 34s	remaining: 52.2s
644:	learn: 0.1385764	total: 1m 34s	remaining: 52s
645:	learn: 0.1384610	total: 1m 34s	remaining: 51.9s
646:	learn: 0.1382732	total: 1m 34s	remaining: 51.7s
647:	learn: 0.1382321	total: 1m 35s	remaining: 51.6s
648:	learn: 0.1381012	total: 1m 35s	remaining: 51.

...

402:	learn: 0.3606395	total: 59s	remaining: 1m 27s
403:	learn: 0.3605479	total: 59.1s	remaining: 1m 27s
404:	learn: 0.3604837	total: 59.3s	remaining: 1m 27s
405:	learn: 0.3603267	total: 59.4s	remaining: 1m 26s
406:	learn: 0.3601669	total: 59.6s	remaining: 1m 26s
407:	learn: 0.3600288	total: 59.7s	remaining: 1m 26s
408:	learn: 0.3599053	total: 59.9s	remaining: 1m 26s
409:	learn: 0.3597075	total: 1m	remaining: 1m 26s
410:	learn: 0.3596219	total: 1m	remaining: 1m 26s
411:	learn: 0.3594956	total: 1m	remaining: 1m 26s
412:	learn: 0.3593303	total: 1m	remaining: 1m 26s
413:	learn: 0.3593142	total: 1m	remaining: 1m 25s
414:	learn: 0.3592778	total: 1m	remaining: 1m 25s
415:	learn: 0.3590694	total: 1m	remaining: 1m 25s
416:	learn: 0.3588999	total: 1m 1s	remaining: 1m 25s
417:	learn: 0.3587941	total: 1m 1s	remaining: 1m 25s
418:	learn: 0.3586402	total: 1m 1s	remaining: 1m 25s
419:	learn: 0.3584672	total: 1m 1s	remaining: 1m 24s
420:	learn: 0.3583251	total: 1m 1s	remaining: 1m 24s
421:	learn: 0.35

556:	learn: 0.3430500	total: 1m 23s	remaining: 1m 6s
557:	learn: 0.3429639	total: 1m 24s	remaining: 1m 6s
558:	learn: 0.3428557	total: 1m 24s	remaining: 1m 6s
559:	learn: 0.3427112	total: 1m 24s	remaining: 1m 6s
560:	learn: 0.3425770	total: 1m 24s	remaining: 1m 6s
561:	learn: 0.3424586	total: 1m 24s	remaining: 1m 6s
562:	learn: 0.3423878	total: 1m 24s	remaining: 1m 5s
563:	learn: 0.3422220	total: 1m 25s	remaining: 1m 5s
564:	learn: 0.3421323	total: 1m 25s	remaining: 1m 5s
565:	learn: 0.3419826	total: 1m 25s	remaining: 1m 5s
566:	learn: 0.3419691	total: 1m 25s	remaining: 1m 5s
567:	learn: 0.3419610	total: 1m 25s	remaining: 1m 5s
568:	learn: 0.3418006	total: 1m 25s	remaining: 1m 5s
569:	learn: 0.3417877	total: 1m 26s	remaining: 1m 4s
570:	learn: 0.3416778	total: 1m 26s	remaining: 1m 4s
571:	learn: 0.3415167	total: 1m 26s	remaining: 1m 4s
572:	learn: 0.3415056	total: 1m 26s	remaining: 1m 4s
573:	learn: 0.3413494	total: 1m 26s	remaining: 1m 4s
574:	learn: 0.3413407	total: 1m 26s	remaining:

712:	learn: 0.3277773	total: 1m 50s	remaining: 44.6s
713:	learn: 0.3276455	total: 1m 51s	remaining: 44.5s
714:	learn: 0.3276083	total: 1m 51s	remaining: 44.4s
715:	learn: 0.3274822	total: 1m 51s	remaining: 44.2s
716:	learn: 0.3274668	total: 1m 51s	remaining: 44.1s
717:	learn: 0.3273032	total: 1m 51s	remaining: 43.9s
718:	learn: 0.3271282	total: 1m 52s	remaining: 43.8s
719:	learn: 0.3271232	total: 1m 52s	remaining: 43.6s
720:	learn: 0.3269174	total: 1m 52s	remaining: 43.5s
721:	learn: 0.3269106	total: 1m 52s	remaining: 43.3s
722:	learn: 0.3268003	total: 1m 52s	remaining: 43.2s
723:	learn: 0.3266077	total: 1m 52s	remaining: 43s
724:	learn: 0.3264628	total: 1m 53s	remaining: 42.9s
725:	learn: 0.3263847	total: 1m 53s	remaining: 42.7s
726:	learn: 0.3262153	total: 1m 53s	remaining: 42.6s
727:	learn: 0.3261392	total: 1m 53s	remaining: 42.5s
728:	learn: 0.3260691	total: 1m 53s	remaining: 42.3s
729:	learn: 0.3259688	total: 1m 54s	remaining: 42.2s
730:	learn: 0.3258552	total: 1m 54s	remaining: 4

869:	learn: 0.3122944	total: 2m 18s	remaining: 20.6s
870:	learn: 0.3121890	total: 2m 18s	remaining: 20.5s
871:	learn: 0.3121150	total: 2m 18s	remaining: 20.3s
872:	learn: 0.3120797	total: 2m 18s	remaining: 20.2s
873:	learn: 0.3119879	total: 2m 18s	remaining: 20s
874:	learn: 0.3119144	total: 2m 18s	remaining: 19.8s
875:	learn: 0.3118488	total: 2m 19s	remaining: 19.7s
876:	learn: 0.3117367	total: 2m 19s	remaining: 19.5s
877:	learn: 0.3115958	total: 2m 19s	remaining: 19.4s
878:	learn: 0.3115542	total: 2m 19s	remaining: 19.2s
879:	learn: 0.3114370	total: 2m 19s	remaining: 19.1s
880:	learn: 0.3113794	total: 2m 19s	remaining: 18.9s
881:	learn: 0.3113742	total: 2m 20s	remaining: 18.7s
882:	learn: 0.3113674	total: 2m 20s	remaining: 18.6s
883:	learn: 0.3112840	total: 2m 20s	remaining: 18.4s
884:	learn: 0.3111619	total: 2m 20s	remaining: 18.3s
885:	learn: 0.3110631	total: 2m 20s	remaining: 18.1s
886:	learn: 0.3110065	total: 2m 20s	remaining: 17.9s
887:	learn: 0.3108653	total: 2m 20s	remaining: 1

OneVsRestClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x7f71382c8350>,
                    n_jobs=None)

In [29]:
#print(clf_multilabel.best_score_)

In [30]:
y_pred = clf_multilabel.predict(x_test)

In [31]:
y_pred

array([[0, 0, 1, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 1, 0, 1],
       ...,
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 1, 1, 1]])

In [32]:
#cm = confusion_matrix(y_test,y_pred) 
#Classification metrics can't handle a mix of multilabel-indicator and unknown targets
print(jaccard_score(y_test,y_pred,average='micro'))
print(jaccard_score(y_test,y_pred,average='macro'))
print(roc_auc_score(y_test,y_pred))
print(hamming_loss(y_test,y_pred))


0.5340983606557377
0.42192158909801875
0.6860483277030485
0.1927433028145134


In [33]:
y_predtest_prob = clf_multilabel.predict_proba(x_test)

In [34]:
y_predtrain_prob = clf_multilabel.predict_proba(x_train)

In [35]:
y_predtrain_prob.shape

(50129, 20)

In [36]:
y_predtrain_prob = pd.DataFrame(y_predtrain_prob)

In [37]:
y_predtrain_prob.shape

(50129, 20)

In [38]:
y_predtest_prob = pd.DataFrame(y_predtest_prob)

In [39]:
type(x_train)

pandas.core.frame.DataFrame

In [40]:
#even train and test set changes everytime as there is random shuffling
#to take into account that as well i have transferred it to pickle
#run this before ml2
x_train.to_pickle('./x_train.pkl')
y_train.to_pickle('./y_train.pkl')
x_test.to_pickle('./x_test.pkl')
y_test.to_pickle('./y_test.pkl')

In [41]:
y_predtest_prob.to_pickle("./y_predtest_probcat.pkl")

In [42]:
y_predtrain_prob.to_pickle("./y_predtrain_probcat.pkl")